<a href="https://colab.research.google.com/github/Dhineshks/Machine_Translation/blob/main/machine_translation_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download and process data from http://www.manythings.org/anki/ 

In [1]:
!wget http://www.manythings.org/anki/hin-eng.zip
!unzip hin-eng.zip

--2021-09-14 12:28:51--  http://www.manythings.org/anki/hin-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.92.44, 172.67.186.54, 2606:4700:3030::6815:5c2c, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.92.44|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 133900 (131K) [application/zip]
Saving to: ‘hin-eng.zip’

hin-eng.zip         100%[===================>] 130.76K  --.-KB/s    in 0.03s   

2021-09-14 12:28:51 (3.80 MB/s) - ‘hin-eng.zip’ saved [133900/133900]

Archive:  hin-eng.zip
  inflating: hin.txt                 
  inflating: _about.txt              


In [2]:
!ls

_about.txt  hin-eng.zip  hin.txt  sample_data
